In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [32]:
from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import AdjToSpTensor, LayerPreprocess
from spektral.data import Dataset
from spektral.transforms.normalize_adj import NormalizeAdj

In [20]:
learning_rate = 1e-2
seed = 0
epochs = 200
patience = 10

In [21]:
tf.random.set_seed(seed=seed)  # make weight initialization reproducible

In [37]:
class NetworkDataset(Dataset):
    def __init__(self, **kwargs):

        super().__init__(**kwargs)
    
    def read(self):
        # We must return a list of Graph objects
        output = []

        data = np.loadtxt(open("output.csv", "rb"), delimiter=",", usecols=(0,2))
        output.append(Graph(e=data))

        return output

In [40]:
# Load data
dataset = NetworkDataset(
    transforms=[LayerPreprocess(GCNConv), AdjToSpTensor()]
)

ValueError: could not convert string to float: 'CUmrqr4svHuSXJy5z7'

In [7]:
# We convert the binary masks to sample weights so that we can compute the
# average loss over the nodes (following original implementation by
# Kipf & Welling)
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)

In [8]:
weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

In [9]:
model = GCN(n_labels=dataset.n_labels, n_input_channels=dataset.n_node_features)
model.compile(
    optimizer=Adam(learning_rate),
    loss=CategoricalCrossentropy(reduction="sum"),
    weighted_metrics=["acc"],
)

In [10]:
# Train model
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/200
1/1 [==============================] - 1s 952ms/step - loss: 1.9528 - acc: 0.1786 - val_loss: 1.9490 - val_acc: 0.2860
Epoch 2/200
1/1 [==============================] - 0s 33ms/step - loss: 1.9473 - acc: 0.3357 - val_loss: 1.9444 - val_acc: 0.4000
Epoch 3/200
1/1 [==============================] - 0s 38ms/step - loss: 1.9402 - acc: 0.4929 - val_loss: 1.9391 - val_acc: 0.4720
Epoch 4/200
1/1 [==============================] - 0s 31ms/step - loss: 1.9321 - acc: 0.6357 - val_loss: 1.9335 - val_acc: 0.5500
Epoch 5/200
1/1 [==============================] - 0s 40ms/step - loss: 1.9227 - acc: 0.6857 - val_loss: 1.9279 - val_acc: 0.6120
Epoch 6/200
1/1 [==============================] - 0s 37ms/step - loss: 1.9145 - acc: 0.7214 - val_loss: 1.9224 - val_acc: 0.6180
Epoch 7/200
1/1 [==============================] - 0s 38ms/step - loss: 1.9045 - acc: 0.7357 - val_loss: 1.9171 - val_acc: 0.6360
Epoch 8/200
1/1 [==============================] - 0s 36ms/step - loss: 1.8995 - acc: 0.7

1/1 [==============================] - 0s 34ms/step - loss: 0.7795 - acc: 0.9571 - val_loss: 1.1939 - val_acc: 0.7980
Epoch 126/200
1/1 [==============================] - 0s 43ms/step - loss: 0.7549 - acc: 0.9571 - val_loss: 1.1911 - val_acc: 0.7980
Epoch 127/200
1/1 [==============================] - 0s 35ms/step - loss: 0.8394 - acc: 0.9286 - val_loss: 1.1885 - val_acc: 0.7940
Epoch 128/200
1/1 [==============================] - 0s 34ms/step - loss: 0.7321 - acc: 0.9429 - val_loss: 1.1854 - val_acc: 0.7960
Epoch 129/200
1/1 [==============================] - 0s 36ms/step - loss: 0.7657 - acc: 0.9286 - val_loss: 1.1825 - val_acc: 0.7960
Epoch 130/200
1/1 [==============================] - 0s 37ms/step - loss: 0.7202 - acc: 0.9571 - val_loss: 1.1796 - val_acc: 0.7940
Epoch 131/200
1/1 [==============================] - 0s 35ms/step - loss: 0.7399 - acc: 0.9429 - val_loss: 1.1764 - val_acc: 0.7940
Epoch 132/200
1/1 [==============================] - 0s 34ms/step - loss: 0.7581 - acc: 0.

In [11]:
# Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 12ms/step - loss: 1.0158 - acc: 0.8230
Done.
Test loss: 1.0157898664474487
Test accuracy: 0.8230001330375671
